In [62]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from google.colab.patches import cv2_imshow
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image



*    **Prediction**



In [63]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import numpy as np
import os
import shutil

In [64]:
def predict(dossier_H, dossier_NH, le_nom_du_psp):
    cpt = 0
    cpt_total = 0
    prediction_list = []
    files_names = []
    Holo = False


    def process_and_predict(image_path):
        nonlocal cpt, cpt_total

        # Create a temporary directory structure
        base_dir = 'temp_base_dir'
        temp_dir = os.path.join(base_dir, 'temp_image_dir')
        if os.path.exists(base_dir):
            shutil.rmtree(base_dir)
        os.makedirs(temp_dir)

        # Copy the specific image to the temporary directory
        temp_image_path = os.path.join(temp_dir, os.path.basename(image_path))
        shutil.copy(image_path, temp_image_path)

        # Define the ImageDataGenerator
        datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

        # Flow from directory
        generator = datagen.flow_from_directory(
            base_dir,
            target_size=(224, 224),
            batch_size=1,
            class_mode=None,
            shuffle=False
        )

        # Get the prediction
        prediction = model.predict(generator)
        y_pred_prob = prediction.squeeze()  # Ensure it's a 1D array

        if y_pred_prob >= 0.5:
            rounded_prediction = 1
        else:
            rounded_prediction = 0

        prediction_list.append(rounded_prediction)
        files_names.append(os.path.basename(image_path))

        if rounded_prediction == 0:
            cpt += 1
        cpt_total += 1

        # Clean up the temporary directory and image
        shutil.rmtree(base_dir)

    if os.path.exists(os.path.join(dossier_H, le_nom_du_psp)) and os.path.exists(os.path.join(dossier_NH, le_nom_du_psp)):
        Holo = True
        for nom_fichier in sorted(os.listdir(os.path.join(dossier_H, le_nom_du_psp))):
            chemin_complet = os.path.join(dossier_H, le_nom_du_psp, nom_fichier)
            if os.path.isfile(chemin_complet):
                process_and_predict(chemin_complet)

        for nom_fichier in sorted(os.listdir(os.path.join(dossier_NH, le_nom_du_psp))):
            chemin_complet = os.path.join(dossier_NH, le_nom_du_psp, nom_fichier)
            if os.path.isfile(chemin_complet):
                process_and_predict(chemin_complet)

    elif not os.path.exists(os.path.join(dossier_H, le_nom_du_psp)) and os.path.exists(os.path.join(dossier_NH, le_nom_du_psp)):
        for nom_fichier in sorted(os.listdir(os.path.join(dossier_NH, le_nom_du_psp))):
            chemin_complet = os.path.join(dossier_NH, le_nom_du_psp, nom_fichier)
            if os.path.isfile(chemin_complet):
                process_and_predict(chemin_complet)
    return prediction_list, files_names,cpt



*   **Color Map creation**



In [65]:
def divide_image_into_patches(image_path, patch_size):
    image =  cv2.imread(image_path)
    height, width = image.shape[:2]
    patch_height, patch_width = patch_size
    patches = []
    cpt = 0
    for y in range(0, height, patch_height):
        for x in range(0, width, patch_width):
            patch = image[y:y+patch_height, x:x+patch_width]
            patches.append(patch)
            cpt+=1
    return patches,cpt

In [66]:
def get_patches_mask(patches_mask):
  labels = []
  for i,patch in enumerate(patches_mask):
    if np.all(patch == 0):
      labels .append('No-Holo')
    else  :
      labels .append('Holo')
  return labels

In [67]:
def get_file_by_index(index, directory):
    # Check if directory exists
    if not os.path.exists(directory) or not os.path.isdir(directory):
        print("Directory does not exist.")
        return None

    # Get list of files in the directory
    files = sorted(os.listdir(directory))

    # Check if index is valid
    if index < 0 or index >= len(files):
        print("Invalid index.")
        return None

    # Get the file corresponding to the index
    file_name = files[index]
    file_path = os.path.join(directory, file_name)
    # Check if file exists (optional step)
    if not os.path.isfile(file_path):
        print("File does not exist.")
        return None
    return file_path,file_name

In [68]:
def extract_patch_from_image(index, directory, patch_size=(40,40 )):
    # Get file path
    file_path,file_name = get_file_by_index(index, directory)
    if not file_path:
        return None

    # Read the image
    try:
        image = cv2.imread(file_path)
        if image is None:
            print("Failed to read the image.")
            return None
    except Exception as e:
        print("Failed to open the image:", e)
        return None

    # Extract patch from top-left corner
    patch = image[0:patch_size[0], 0:patch_size[1]]

    return patch,file_name

In [69]:
def add_tint(image, color, intensity):
    # Create a solid color image with the specified color and same size as the original image
    color_image = np.zeros_like(image)
    color_image[:] = color

    # Blend the original image with the solid color image using the specified intensity
    tinted_image = cv2.addWeighted(image, 1 - intensity, color_image, intensity, 0)

    return tinted_image

In [70]:
def create_patch_grid(patches, num_columns=8, num_rows=5):
    # Get dimensions of each patch
    patch_height, patch_width, _ = patches[0].shape

    # Calculate the total number of patches
    total_patches = min(len(patches), num_columns * num_rows)

    # Calculate the size of the grid
    grid_height = num_rows * patch_height
    grid_width = num_columns * patch_width

    # Create blank image to hold patches
    grid = np.zeros((grid_height, grid_width, 3), dtype=np.uint8)

    # Paste patches onto the blank image
    for i in range(total_patches):
        row = i // num_columns
        col = i % num_columns
        y_start = row * patch_height
        y_end = y_start + patch_height
        x_start = col * patch_width
        x_end = x_start + patch_width
        grid[y_start:y_end, x_start:x_end] = patches[i]

    return grid

In [71]:
def recreat_psp_with_tint(labels,dossier_H,le_nom_du_psp,files_names):
  holo_cpt = 0
  no_holo_cpt = 0
  image = []
  if os.path.exists(dossier_H+le_nom_du_psp) :
    for label in labels :
      if label == "Holo" :
          patch, file_name = extract_patch_from_image(holo_cpt, dossier_H + le_nom_du_psp)
          holo_cpt +=1
          index = files_names.index(file_name)
          if prediction_list[index] == 0 :
            image.append(add_tint(patch,(0, 255, 0), 1))
          else:
            image.append(add_tint(patch,(0, 0, 255),1))
      if label =="No-Holo":
        patch, file_name = extract_patch_from_image(no_holo_cpt, dossier_NH + le_nom_du_psp)
        no_holo_cpt +=1
        index = files_names.index(file_name)
        if prediction_list[index] == 0 :
          image.append(add_tint(patch,(0, 255, 0), 1))
        else:
          image.append(add_tint(patch,(0, 0, 255), 1))
  else:
    patch_holo = []
    patch_no_holo = []
    for label in labels :
      if label == "Holo" :
          patch, file_name = extract_patch_from_image(no_holo_cpt, dossier_NH + le_nom_du_psp)
          no_holo_cpt +=1
          index = files_names.index(file_name)
          if prediction_list[index] == 0 :
            patch_holo.append(add_tint(patch,(0, 255, 0), 1))
          else:
            patch_holo.append(add_tint(patch,(0, 0, 255), 1))
    for label in labels :
        if label =="No-Holo":
          patch, file_name = extract_patch_from_image(no_holo_cpt, dossier_NH + le_nom_du_psp)
          no_holo_cpt +=1
          index = files_names.index(file_name)
          if prediction_list[index] == 0 :
            patch_no_holo.append(add_tint(patch,(0, 255, 0), 1))
          else:
            patch_no_holo.append(add_tint(patch,(0, 0, 255), 1))
    cpt_holo = 0
    cpt_no_holo = 0
    for label in labels :
        if label == "Holo" :
            image.append(patch_holo[cpt_holo])
            cpt_holo += 1
        else :
            image.append(patch_no_holo[cpt_no_holo])
            cpt_no_holo += 1
  return image

In [72]:
model = tf.keras.models.load_model('/content/drive/MyDrive/TER Data/final best model/best_model.h5')

In [73]:
mask_path = "/content/drive/MyDrive/TER Data/templates/hologram_masks/passport_hologram_mask_600_400.png"
patch_size = (40,40)
patches_mask, num_patches = divide_image_into_patches(mask_path, patch_size)
labels =  get_patches_mask(patches_mask)

In [74]:
dossier_H = "/content/drive/MyDrive/TER Data/test/Holo/"
dossier_NH = "/content/drive/MyDrive/TER Data/test/No-Holo/"

In [75]:
psp_names = []
final_labels = []
for path in [dossier_H,dossier_NH]:
  for item in os.listdir(path):
          if os.path.isdir(os.path.join(path, item)) and item not in psp_names:
              psp_names.append(item)
              if path ==  dossier_H :
                final_labels.append('Holo')
              else:
                final_labels.append('No-Holo')

In [76]:
save_folder = "/content/drive/MyDrive/TER Data/final best color maps 40x40"
cpt = 0
tot=0
for le_nom_du_psp in psp_names :
  prediction_list,files_names,res = predict(dossier_H,dossier_NH,le_nom_du_psp)
  image = recreat_psp_with_tint(labels,dossier_H,le_nom_du_psp,files_names)
  grid_image = create_patch_grid(image, num_columns=15, num_rows=10)
  if final_labels[psp_names.index(le_nom_du_psp)] == "Holo":
    cv2.imwrite(save_folder + "/Holo/"+le_nom_du_psp+'.png', grid_image)
  else:
    cv2.imwrite(save_folder + "/No-Holo/" + le_nom_du_psp+'.png', grid_image)
  tot+=res
  print(cpt)
  cpt+=1
print(tot)

Streaming output truncated to the last 5000 lines.
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 69ms/step
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 96ms/step
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 117ms/step
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 101ms/step
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 97ms/step
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 97ms/step
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 107ms/step
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 100ms/step
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 96ms/step
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 104ms/step
Found 1 images belonging to 1 clas